**Guilherme Augusto da Silva**

**RM 354300**

### **DEFINIÇÃO DO PROBLEMA**
O problema do investidor com as características moderadora é encontrar uma carteira de investimentos onde se minimiza o risco, mas sem desconsiderar um bom retorno. Objetivo: Linha tênue entre minimizar o risco e maximizar o lucro!

**Bibliotecas e Parâmetros Iniciais**

In [35]:
import numpy as np

num_assets = 5
population_size = 20
elite_size = 1
tournament_size = 3
target_sharpe = 2.0
mutation_rate = 0.01
max_generations = 3

historical_returns = np.array([
    [0.01, 0.02, 0.015, -0.005, 0.007],
    [0.012, 0.018, 0.016, -0.004, 0.009],
    [0.015, 0.017, 0.014, -0.006, 0.008],
    [0.013, 0.019, 0.015, -0.003, 0.007],
    [0.014, 0.016, 0.013, -0.002, 0.009],
    # ...
])
risk_free_rate = 0.01
target_sharpe = 15

Importamos a biblioteca numpy para manipulação de arrays e cálculos. Definimos o número de ativos, tamanho da populado e histórico dos retornos, além da taxa de retorno livre do risco.

**Gera População Inicial**

In [36]:
def generate_portfolio(num_assets):
    weights = np.random.rand(num_assets)
    weights /= np.sum(weights)
    return weights

Gera uma carteira de investimentos com pesos aleatórios para cada ativo, normalizados para que a soma dos pesos seja 1. (100% do capital)

**Função fitness**

In [37]:
def fitness_function(weights, returns, risk_free_rate):
    portfolio_return = np.sum(returns.mean(axis=0) * weights)
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(np.cov(returns.T), weights)))
    if portfolio_std_dev != 0:  # Evita divisão por zero
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    else:
        sharpe_ratio = 0
    return sharpe_ratio


Função de aptidão calcula o índice de Sharpe, que é uma fórmula de retorno ajustado pelo risco da carteira.

O Índice de Sharpe é definido como:
Sharpe Ratio
=
(𝐸
(𝑅𝑝)
−
𝑅𝑓) /
𝜎𝑝

Onde:

E(𝑅𝑝)
é o retorno esperado da carteira.
𝑅𝑓
é a taxa de retorno livre de risco.
𝜎𝑝
é o desvio padrão do retorno da carteira (medida de risco).
O Índice de Sharpe indica quanto retorno em excesso (acima do retorno livre de risco) é obtido por unidade de risco. Quanto maior o Índice de Sharpe, melhor é o retorno ajustado pelo risco da carteira.

**Geração da População Inicial e Cálculo da Aptidão**

In [38]:
population = [generate_portfolio(num_assets) for _ in range(population_size)]

for i, portfolio in enumerate(population):
    fitness = fitness_function(portfolio, historical_returns, risk_free_rate)
    print(f"Carteira {i + 1}: {portfolio}, Fitness (Índice de Sharpe): {fitness}")


Carteira 1: [0.21120341 0.11202615 0.27511508 0.09563171 0.30602365], Fitness (Índice de Sharpe): 1.8943951636174567
Carteira 2: [0.25387818 0.28600805 0.08981163 0.30134229 0.06895985], Fitness (Índice de Sharpe): -2.1050373726087646
Carteira 3: [0.30145089 0.05709528 0.15383083 0.18369291 0.30393008], Fitness (Índice de Sharpe): -1.703242046171541
Carteira 4: [0.04703683 0.30765589 0.29512779 0.32027488 0.0299046 ], Fitness (Índice de Sharpe): -0.8542971686396891
Carteira 5: [0.34657524 0.13822131 0.31342816 0.09034924 0.11142605], Fitness (Índice de Sharpe): 4.442502811241238
Carteira 6: [0.14816222 0.02722158 0.31174626 0.18070996 0.33215998], Fitness (Índice de Sharpe): -2.105313469738447
Carteira 7: [0.17074728 0.17148603 0.24781355 0.18243059 0.22752255], Fitness (Índice de Sharpe): -0.05131528454964821
Carteira 8: [0.13795651 0.31093605 0.25655795 0.03668089 0.25786859], Fitness (Índice de Sharpe): 7.509218382127829
Carteira 9: [0.53638331 0.03182964 0.26816352 0.10681557 0.056

Aqui, geramos a população inicial de carteiras e calculamos o Índice de Sharpe para cada uma, que serve como medida de aptidão. Isso nos permite avaliar o desempenho inicial das carteiras.

**Seleção por Torneio e Crossover Uniforme**

In [39]:
def tournament_selection(population, fitness_scores, k):
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = [(i, fitness_scores[i]) for i in selected_indices]
    selected = sorted(selected, key=lambda x: x[1], reverse=True)
    return population[selected[0][0]]

def uniform_crossover(parent1, parent2):
    mask = np.random.rand(len(parent1)) < 0.5
    child = np.where(mask, parent1, parent2)
    return child


A seleção por torneio escolhe os indivíduos para reprodução com maior aptidão. O crossover uniforme gera novos filhos combinando genes dos pais de forma aleatória, promovendo diversidade genética.

**Mutação Gaussiana**

In [40]:
def gaussian_mutation(weights, mutation_rate):
    return weights + np.random.normal(0, mutation_rate, len(weights))


A mutação gaussiana adiciona um valor aleatório a cada peso da carteira, baseado em uma distribuição normal.

**Evolução e Substituição da População**

In [41]:
generation = 0
while generation < max_generations:
    fitness_scores = np.array([fitness_function(portfolio, historical_returns, risk_free_rate) for portfolio in population])

    max_fitness = np.max(fitness_scores)
    if max_fitness >= target_sharpe:
        break

    elite_indices = np.argsort(fitness_scores)[-elite_size:]
    elite_individuals = [population[i] for i in elite_indices]

    new_population = elite_individuals.copy()
    while len(new_population) < population_size:
        parent1 = tournament_selection(population, fitness_scores, tournament_size)
        parent2 = tournament_selection(population, fitness_scores, tournament_size)
        child = uniform_crossover(parent1, parent2)
        child = gaussian_mutation(child, mutation_rate)
        child /= np.sum(child)
        new_population.append(child)

    population = new_population
    generation += 1

Neste loop, a aptidão de cada carteira é calculada e a melhor aptidão é comparada com um alvo. Se o alvo é atingido, o processo é encerrado. Caso contrário, uma nova população é gerada através de elitismo, seleção por torneio, crossover e mutação. A nova população substitui a antiga, e o ciclo continua até que a condição de término seja atendida.

**Resultado Final**

In [42]:
for i, portfolio in enumerate(population):
    fitness = fitness_function(portfolio, historical_returns, risk_free_rate)
    print(f"Carteira {i + 1}: {portfolio}, Fitness (Índice de Sharpe): {fitness}")

print(f"Evolução concluída em {generation} gerações com melhor Índice de Sharpe: {max_fitness}")


Carteira 1: [0.20351683 0.37048398 0.15515936 0.04178074 0.22905909], Fitness (Índice de Sharpe): 10.675588349320751
Carteira 2: [0.19301415 0.33930295 0.14675107 0.02549374 0.2954381 ], Fitness (Índice de Sharpe): 11.300283264908455
Carteira 3: [0.25650355 0.55558067 0.03052648 0.03062154 0.12676776], Fitness (Índice de Sharpe): 10.4491900027609
Carteira 4: [0.2766161  0.29955801 0.2030635  0.03216943 0.18859297], Fitness (Índice de Sharpe): 10.764568615237888
Carteira 5: [0.32400422 0.32419919 0.25412999 0.0301075  0.06755911], Fitness (Índice de Sharpe): 10.463365258630793
Carteira 6: [0.12810805 0.32912773 0.25659586 0.03042962 0.25573874], Fitness (Índice de Sharpe): 7.4360760797085765
Carteira 7: [0.35156044 0.34327264 0.20366653 0.02635334 0.07514705], Fitness (Índice de Sharpe): 10.840895908873966
Carteira 8: [0.21490164 0.33912054 0.16824608 0.03721122 0.24052052], Fitness (Índice de Sharpe): 11.032989682100002
Carteira 9: [0.2165322  0.32345626 0.17299602 0.03700335 0.2500121

**Conclusões**

As carteiras geradas pelo algoritmo genético mostram uma diversificação razoável entre os ativos, indicando um bom balanceamento de risco e retorno. Apesar de o algoritmo ter evoluído por apenas três gerações, os Índices de Sharpe permaneceram consistentemente altos, sugerindo que o modelo atingiu rapidamente um ponto de saturação ou que o espaço de busca é limitado. É crucial verificar se os dados históricos de retornos são realistas e se os parâmetros do modelo estão adequados para evitar superestimações e garantir que os resultados sejam representativos de condições reais de mercado.